In [1]:
import psycopg2
import random
from collections import defaultdict


In [2]:
def print_table(cursor, table_name):
    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    print(f"Table {table_name}")
    for row in rows:
        print(row)
    print()

In [3]:


# a: Verbindung zur Datenbank herstellen
try:
    connection = psycopg2.connect(
        dbname="Sparsity", user="postgres", password="", host="localhost", port="5432"
    )
    cursor = connection.cursor()
    cursor.execute("SELECT version();")
    print(cursor.fetchone())
except (Exception, psycopg2.Error) as error:
    print("Error while connecting to PostgreSQL", error)

('PostgreSQL 16.8 on x86_64-redhat-linux-gnu, compiled by gcc (GCC) 14.2.1 20250110 (Red Hat 14.2.1-7), 64-bit',)


In [4]:
cursor.execute(
    """
    DROP TABLE IF EXISTS H_toy CASCADE;
    DROP TABLE IF EXISTS V_toy CASCADE;
    DROP TABLE IF EXISTS V_toy_string CASCADE;
    DROP TABLE IF EXISTS V_toy_int CASCADE;
    DROP TABLE IF EXISTS H CASCADE;
    DROP VIEW IF EXISTS h2v_toy CASCADE;
    DROP VIEW IF EXISTS V_toy_all CASCADE;
    DROP VIEW IF EXISTS H_non_null CASCADE;
    DROP VIEW IF EXISTS H_null_count CASCADE;
"""
)
connection.commit()


In [5]:

# b: Tabelle H_toy erstellen und befüllen
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS H_toy (
        oid SERIAL PRIMARY KEY,
        a1 VARCHAR(50),
        a2 VARCHAR(50),
        a3 INTEGER
    );
"""
)
cursor.execute(
    """
    INSERT INTO H_toy (a1, a2, a3) VALUES
    ('a', 'b', NULL),
    (NULL, 'c', 2),
    (NULL, NULL, 3),
    (NULL, NULL, NULL);
"""
)
connection.commit()

# Vertikale Darstellung V_toy erstellen
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS V_toy (
        oid SERIAL PRIMARY KEY,
        key VARCHAR(50),
        val VARCHAR(50)
    );
"""
)
cursor.execute(
    """
    INSERT INTO V_toy (oid, key, val) VALUES
    (11, 'a1', 'a'),
    (12, 'a2', 'b'),
    (21, 'a2', 'c'),
    (22, 'a3', '2'),
    (31, 'a3', '3'),
    (41, 'ai', 'z');
"""
)
connection.commit()

print_table(cursor, "H_toy")
print_table(cursor, "V_toy")

Table H_toy
(1, 'a', 'b', None)
(2, None, 'c', 2)
(3, None, None, 3)
(4, None, None, None)

Table V_toy
(11, 'a1', 'a')
(12, 'a2', 'b')
(21, 'a2', 'c')
(22, 'a3', '2')
(31, 'a3', '3')
(41, 'ai', 'z')



In [6]:

# Sicht h2v_toy erstellen
cursor.execute(
    """
    CREATE VIEW h2v_toy AS
    SELECT 
        base.oid AS oid,
        v1.val AS a1,
        v2.val AS a2,
        v3.val AS a3
    FROM 
        (SELECT DISTINCT oid FROM V_toy) AS base
    LEFT JOIN 
        V_toy AS v1 ON base.oid = v1.oid AND v1.key = 'a1'
    LEFT JOIN 
        V_toy AS v2 ON base.oid = v2.oid AND v2.key = 'a2'
    LEFT JOIN 
        V_toy AS v3 ON base.oid = v3.oid AND v3.key = 'a3';
"""
)
connection.commit()

print_table(cursor, "h2v_toy")

Table h2v_toy
(22, None, None, '2')
(31, None, None, '3')
(11, 'a', None, None)
(21, None, 'c', None)
(41, None, None, None)
(12, None, 'b', None)



In [7]:

# Partitionen erstellen
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS V_toy_string (
        oid INTEGER,
        key VARCHAR(50),
        val VARCHAR(50)
    );
"""
)
cursor.execute(
    """
    CREATE TABLE IF NOT EXISTS V_toy_int (
        oid INTEGER,
        key VARCHAR(50),
        val INTEGER
    );
"""
)
connection.commit()

# Daten in Partitionen einfügen
cursor.execute(
    """
    INSERT INTO V_toy_string (oid, key, val) VALUES
    (1, 'a1', 'a'),
    (1, 'a2', 'b'),
    (2, 'a2', 'c'),
    (4, 'ai', '');
"""
)
cursor.execute(
    """
    INSERT INTO V_toy_int (oid, key, val) VALUES
    (2, 'a3', 2),
    (3, 'a3', 3),
    (4, 'ai', -1);
"""
)
connection.commit()

print_table(cursor, "V_toy_string")
print_table(cursor, "V_toy_int")

Table V_toy_string
(1, 'a1', 'a')
(1, 'a2', 'b')
(2, 'a2', 'c')
(4, 'ai', '')

Table V_toy_int
(2, 'a3', 2)
(3, 'a3', 3)
(4, 'ai', -1)



In [8]:

# Sicht V_toy_all erstellen
cursor.execute(
    """
    CREATE VIEW V_toy_all AS
    SELECT oid, key, val::VARCHAR(50) AS val FROM V_toy_string
    UNION ALL
    SELECT oid, key, val::VARCHAR(50) FROM V_toy_int;
"""
)
connection.commit()

print_table(cursor, "V_toy_all")

Table V_toy_all
(1, 'a1', 'a')
(1, 'a2', 'b')
(2, 'a2', 'c')
(4, 'ai', '')
(2, 'a3', '2')
(3, 'a3', '3')
(4, 'ai', '-1')



In [9]:

# c: Korrigierte Funktion generate()
def generate(num_tuples, sparsity, num_attributes):
    try:
        # Alte Tabelle löschen, falls vorhanden
        cursor.execute("DROP TABLE IF EXISTS H;")

        # Tabelle H erstellen
        columns = ["oid SERIAL PRIMARY KEY"]
        for i in range(1, num_attributes + 1):
            if i % 2 == 0:
                columns.append(f"a{i} INTEGER")
            else:
                columns.append(f"a{i} VARCHAR(50)")
        create_table_query = f"CREATE TABLE H ({', '.join(columns)});"
        cursor.execute(create_table_query)

        # Begrenzung für Attributwerte (maximal 5-mal vorkommen)
        value_counts = defaultdict(int)
        allowed_values = ["a", "b", "c"]

        # Daten einfügen
        for oid in range(1, num_tuples + 1):
            values = []
            for i in range(1, num_attributes + 1):
                if random.random() < sparsity:
                    values.append("NULL")
                else:
                    if i % 2 == 0:
                        values.append(str(random.randint(1, 100)))
                    else:
                        # Begrenzung auf max. 5 Vorkommen pro Wert
                        valid_choices = [
                            v for v in allowed_values if value_counts[v] < num_attributes * num_tuples/2*(1-sparsity)
                        ]
                        if valid_choices:
                            chosen_value = random.choice(valid_choices)
                            values.append(f"'{chosen_value}'")
                            value_counts[chosen_value] += 1
                        else:
                            values.append("'z'")
            insert_query = f"INSERT INTO H ({', '.join([f'a{i}' for i in range(1, num_attributes + 1)] )}) VALUES ({', '.join(values)});"
            cursor.execute(insert_query)

        connection.commit()
        print(
            f"Tabelle H mit {num_tuples} Tupeln und {num_attributes} Attributen erstellt."
        )
    except (Exception, psycopg2.Error) as error:
        print("Fehler in generate():", error)


In [10]:


# Beispielaufruf der Funktion generate()
generate(num_tuples=20, sparsity=0.3, num_attributes=5)

# Zusätzliche Sichten zur Prüfung der Korrektheit
# Vorhandene Sichten löschen, falls sie existieren
cursor.execute("DROP VIEW IF EXISTS H_non_null;")
cursor.execute("DROP VIEW IF EXISTS H_null_count;")
connection.commit()

# Sichten zur Überprüfung der Korrektheit
cursor.execute(
    """
    CREATE VIEW H_null_count AS
    SELECT COUNT(*) FROM H WHERE a1 IS NULL OR a2 IS NULL OR a3 IS NULL OR a4 IS NULL OR a5 IS NULL;
"""
)
connection.commit()

print_table(cursor, "H")
print_table(cursor, "H_null_count")



Tabelle H mit 20 Tupeln und 5 Attributen erstellt.
Table H
(1, None, 33, 'a', 93, None)
(2, 'a', None, 'b', 13, 'b')
(3, None, None, None, None, 'a')
(4, 'a', 62, 'c', None, 'a')
(5, 'b', None, None, 30, 'a')
(6, 'a', None, None, 78, 'a')
(7, 'c', 66, None, 93, 'a')
(8, 'b', 21, 'b', None, 'a')
(9, 'c', 41, 'b', 74, None)
(10, 'a', 32, 'b', 2, 'a')
(11, 'c', 19, None, 16, 'c')
(12, 'c', 52, 'a', 81, 'b')
(13, None, 14, 'b', 96, 'b')
(14, 'c', 77, 'c', 36, None)
(15, None, 28, 'a', 87, 'a')
(16, 'b', 42, 'a', None, None)
(17, 'b', 41, None, 33, 'b')
(18, 'a', 39, 'b', 41, None)
(19, 'c', 49, 'c', 31, 'c')
(20, 'b', 63, 'b', None, 'b')

Table H_null_count
(17,)

